# Likelihood Revisit

In [173]:
import numpy as np


def conditional_distribution(mean, cov, stat):
    """ This function calculates the conditional distribution This function assumes that the draws refer to the
    last elements."""
    
    q, p = len(mean) - len(stat), len(stat)
    
    
    mu_1, mu_2 = mean[0:q], mean[q:(p + q)]

    sigma_11 = cov[0:q,0:q]
    sigma_22 = cov[q:(q + p), q:(q + p)]
    
    
    sigma_12 = cov[0:q, q:(q + p)]
    sigma_21 = cov[q:(q + p),0:q]
    
    
    # Ensuring that the simple decompositions worked properly
    upper = np.hstack((sigma_11, sigma_12))
    lower = np.hstack((sigma_21, sigma_22))

    np.testing.assert_equal(np.hstack((mu_1, mu_2)), mean)
    np.testing.assert_equal(np.vstack((upper, lower)), cov)
    

    cond_mean = mu_1 + np.matmul(np.matmul(sigma_12, np.linalg.inv(sigma_22)), (stat - mu_2))
    cond_cov = sigma_11 - np.matmul(np.matmul(sigma_12, np.linalg.inv(sigma_22)), sigma_21)
    
    return cond_mean, cond_cov
    

def swap_rows(v, i, j):
    if len(v.shape) == 1:
        v[i], v[j] = v[j], v[i]
    else:
        v[[i, j], :] = v[[j, i], :]

    return v

def swap_columns(v, i, j):
    v[:, [i, j]] = v[:, [j, i]]
    return v
    
np.random.seed(1223)
for _ in range(1):

    num_draws = 4#np.random.randint(low=2, high=10)
    num_stat = 1#np.random.randint(low=1, high=num_draws)

    pos = np.random.choice(range(num_draws), size=num_stat, replace=False)
       
    
    mean = np.array(np.random.uniform(size=num_draws), ndmin=1)
    from scipy.stats import wishart, chi2
    
    cov = wishart.rvs(df=num_draws, scale=np.identity(num_draws))
    
    cov_base = cov.copy()

    for i, j in enumerate(pos):
        mean = swap_rows(mean, -(i + 1), j)
    
    for i, j in enumerate(pos):
        cov_base = swap_columns(cov_base, -(i + 1), j)
        cov_base = swap_rows(cov_base, -(i + 1), j)

        
    base_idx = np.array(range(num_draws), ndmin=1)
    print(base_idx)
    for i, j in enumerate(pos):
        cov = swap_rows(cov, -(i + 1), j)
        cov = swap_columns(cov, -(i + 1), j)
        print( -(i + 1), j)
        base_idx = swap_rows(base_idx, -(i + 1), j)
        
    print(pos, base_idx)
    
    np.testing.assert_equal(cov, cov_base)
    
    stat = np.array(np.random.uniform(size=num_stat), ndmin=1)
    rslt = conditional_distribution(mean, cov, stat)





[0 1 2 3]
-1 1
[1] [0 3 2 1]
